# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [29]:
import os

Build binaries.

In [30]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [31]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [32]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

fatal: destination path 'superchain-registry' already exists and is not an empty directory.
HEAD is now at 5d3ca19 superchain/extra: include minified genesis state data (#2)


Select your chain.

In [33]:
%%capture
%env L1_CHAIN_NAME=goerli
%env L2_CHAIN_NAME=op

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [34]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [35]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


env: L1_START=9500000
env: L1_END=9520000
env: BATCH_TX_SENDER=0x7431310e026B69BFC676C0013E12A1A11411EEc9
env: L2_GENESIS_TIMESTAMP=1673550516
env: L2_CHAIN_ID=420
env: INBOX=0xff00000000000000000000000000000000000420
env: ACC_L2_START=13630000
env: ACC_L2_END=13631000


Get L1 RPC from secret.

In [36]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [37]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [38]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

env: DB_PREFIX=./artifacts/goerli/op


batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [11]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

Fetched block:  9519999
Fetched block:  9519985
Fetched block:  9519991
Fetched block:  9519981
Fetched block:  9519996
Fetched block:  9519998
Fetched block:  9519994
Fetched batches in range [9500000,9520000). Found 5227 valid & 190 invalid batches
Fetch Config: Chain ID: 5. Inbox Address: 0xff00000000000000000000000000000000000420. Valid Senders: map[0x7431310e026B69BFC676C0013E12A1A11411EEc9:{}].
Wrote transactions with batches to ./artifacts/goerli/op/transactions_cache


batch decoder `reassemble`: Reassemble channel frames to channels. 

In [12]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [13]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[4860/4869] Channel ID: ffab1bed61b89d30899f4fec4b2e7f90, L2StartNum: 13197422, L2EndNum: 13197451, L2BlockCnt, 30
[4861/4869] Channel ID: ffc734de488d29782ad9988444dc6eee, L2StartNum: 13215907, L2EndNum: 13215931, L2BlockCnt, 25
[4862/4869] Channel ID: ffc83233adadd732c80a11ea9cf68a9b, L2StartNum: 13217325, L2EndNum: 13217354, L2BlockCnt, 30
[4863/4869] Channel ID: ffceeaa964d38ae197f0c409b01a7f98, L2StartNum: 13177747, L2EndNum: 13177789, L2BlockCnt, 43
[4864/4869] Channel ID: ffd728e29b0ac438d48c31c894753eb1, L2StartNum: 13222640, L2EndNum: 13222686, L2BlockCnt, 47
[4865/4869] Channel ID: ffdf70003c3c6cf4e52aa4e8887a65f1, L2StartNum: 13187472, L2EndNum: 13187501, L2BlockCnt, 30
[4866/4869] Channel ID: ffe558cbf7706335fada84d3da3e5c95, L2StartNum: 13291435, L2EndNum: 13291474, L2BlockCnt, 40
[4867/4869] Channel ID: fffc9f6ea9b63b33c5c17d0fb1a7061f, L2StartNum: 13174502, L2EndNum: 13174524, L2BlockCnt, 23
[4868/4869] Channel ID: fffd5a17a99e98edf732f21d8f406fbb, L2StartNum: 13221157, 

In [14]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[4860/4869] Channel ID: ffab1bed61b89d30899f4fec4b2e7f90, L2StartNum: 13197422, L2EndNum: 13197451, L2BlockCnt, 30
[4861/4869] Channel ID: ffc734de488d29782ad9988444dc6eee, L2StartNum: 13215907, L2EndNum: 13215931, L2BlockCnt, 25
[4862/4869] Channel ID: ffc83233adadd732c80a11ea9cf68a9b, L2StartNum: 13217325, L2EndNum: 13217354, L2BlockCnt, 30
[4863/4869] Channel ID: ffceeaa964d38ae197f0c409b01a7f98, L2StartNum: 13177747, L2EndNum: 13177789, L2BlockCnt, 43
[4864/4869] Channel ID: ffd728e29b0ac438d48c31c894753eb1, L2StartNum: 13222640, L2EndNum: 13222686, L2BlockCnt, 47
[4865/4869] Channel ID: ffdf70003c3c6cf4e52aa4e8887a65f1, L2StartNum: 13187472, L2EndNum: 13187501, L2BlockCnt, 30
[4866/4869] Channel ID: ffe558cbf7706335fada84d3da3e5c95, L2StartNum: 13291435, L2EndNum: 13291474, L2BlockCnt, 40
[4867/4869] Channel ID: fffc9f6ea9b63b33c5c17d0fb1a7061f, L2StartNum: 13174502, L2EndNum: 13174524, L2BlockCnt, 23
[4868/4869] Channel ID: fffd5a17a99e98edf732f21d8f406fbb, L2StartNum: 13221157, 

In [15]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[4860/4869] Channel ID: ffab1bed61b89d30899f4fec4b2e7f90, L2StartNum: 13197422, L2EndNum: 13197451, L2BlockCnt, 30
[4861/4869] Channel ID: ffc734de488d29782ad9988444dc6eee, L2StartNum: 13215907, L2EndNum: 13215931, L2BlockCnt, 25
[4862/4869] Channel ID: ffc83233adadd732c80a11ea9cf68a9b, L2StartNum: 13217325, L2EndNum: 13217354, L2BlockCnt, 30
[4863/4869] Channel ID: ffceeaa964d38ae197f0c409b01a7f98, L2StartNum: 13177747, L2EndNum: 13177789, L2BlockCnt, 43
[4864/4869] Channel ID: ffd728e29b0ac438d48c31c894753eb1, L2StartNum: 13222640, L2EndNum: 13222686, L2BlockCnt, 47
[4865/4869] Channel ID: ffdf70003c3c6cf4e52aa4e8887a65f1, L2StartNum: 13187472, L2EndNum: 13187501, L2BlockCnt, 30
[4866/4869] Channel ID: ffe558cbf7706335fada84d3da3e5c95, L2StartNum: 13291435, L2EndNum: 13291474, L2BlockCnt, 40
[4867/4869] Channel ID: fffc9f6ea9b63b33c5c17d0fb1a7061f, L2StartNum: 13174502, L2EndNum: 13174524, L2BlockCnt, 23
[4868/4869] Channel ID: fffd5a17a99e98edf732f21d8f406fbb, L2StartNum: 13221157, 

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [16]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[4860/4869] Channel ID: ffab1bed61b89d30899f4fec4b2e7f90, CompressedReductionPercent: 2.190540 %
[4861/4869] Channel ID: ffc734de488d29782ad9988444dc6eee, CompressedReductionPercent: 35.468565 %
[4862/4869] Channel ID: ffc83233adadd732c80a11ea9cf68a9b, CompressedReductionPercent: 16.787122 %
[4863/4869] Channel ID: ffceeaa964d38ae197f0c409b01a7f98, CompressedReductionPercent: 21.589404 %
[4864/4869] Channel ID: ffd728e29b0ac438d48c31c894753eb1, CompressedReductionPercent: 32.577132 %
[4865/4869] Channel ID: ffdf70003c3c6cf4e52aa4e8887a65f1, CompressedReductionPercent: 27.126020 %
[4866/4869] Channel ID: ffe558cbf7706335fada84d3da3e5c95, CompressedReductionPercent: 41.796875 %
[4867/4869] Channel ID: fffc9f6ea9b63b33c5c17d0fb1a7061f, CompressedReductionPercent: 35.384125 %
[4868/4869] Channel ID: fffd5a17a99e98edf732f21d8f406fbb, CompressedReductionPercent: 29.637527 %
[4869/4869] Channel ID: fffe3db33486f9a289378384e0d8985a, CompressedReductionPercent: 19.413308 %


In [17]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[4860/4869] Channel ID: ffab1bed61b89d30899f4fec4b2e7f90, CompressedReductionPercent: 2.436590 %
[4861/4869] Channel ID: ffc734de488d29782ad9988444dc6eee, CompressedReductionPercent: 34.282325 %
[4862/4869] Channel ID: ffc83233adadd732c80a11ea9cf68a9b, CompressedReductionPercent: 15.768725 %
[4863/4869] Channel ID: ffceeaa964d38ae197f0c409b01a7f98, CompressedReductionPercent: 20.183393 %
[4864/4869] Channel ID: ffd728e29b0ac438d48c31c894753eb1, CompressedReductionPercent: 31.624319 %
[4865/4869] Channel ID: ffdf70003c3c6cf4e52aa4e8887a65f1, CompressedReductionPercent: 25.602901 %
[4866/4869] Channel ID: ffe558cbf7706335fada84d3da3e5c95, CompressedReductionPercent: 40.280331 %
[4867/4869] Channel ID: fffc9f6ea9b63b33c5c17d0fb1a7061f, CompressedReductionPercent: 34.140899 %
[4868/4869] Channel ID: fffd5a17a99e98edf732f21d8f406fbb, CompressedReductionPercent: 27.955461 %
[4869/4869] Channel ID: fffe3db33486f9a289378384e0d8985a, CompressedReductionPercent: 18.244694 %


In [18]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[4860/4869] Channel ID: ffab1bed61b89d30899f4fec4b2e7f90, CompressedReductionPercent: 3.427816 %
[4861/4869] Channel ID: ffc734de488d29782ad9988444dc6eee, CompressedReductionPercent: 36.773428 %
[4862/4869] Channel ID: ffc83233adadd732c80a11ea9cf68a9b, CompressedReductionPercent: 17.597459 %
[4863/4869] Channel ID: ffceeaa964d38ae197f0c409b01a7f98, CompressedReductionPercent: 22.424860 %
[4864/4869] Channel ID: ffd728e29b0ac438d48c31c894753eb1, CompressedReductionPercent: 33.151845 %
[4865/4869] Channel ID: ffdf70003c3c6cf4e52aa4e8887a65f1, CompressedReductionPercent: 28.467815 %
[4866/4869] Channel ID: ffe558cbf7706335fada84d3da3e5c95, CompressedReductionPercent: 43.014706 %
[4867/4869] Channel ID: fffc9f6ea9b63b33c5c17d0fb1a7061f, CompressedReductionPercent: 36.531718 %
[4868/4869] Channel ID: fffd5a17a99e98edf732f21d8f406fbb, CompressedReductionPercent: 31.390666 %
[4869/4869] Channel ID: fffe3db33486f9a289378384e0d8985a, CompressedReductionPercent: 20.283806 %


span batch tester `fetch`: Fetch consecutive singular batches.

In [19]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=100 --out=$DB_PREFIX/singular_batches_cache | tail -10

Fetched L2 block:  13630990
Fetched L2 block:  13630988
Fetched L2 block:  13630992
Fetched L2 block:  13630997
Fetched L2 block:  13630995
Fetched L2 block:  13630996
Fetched L2 block:  13630999
Fetched L2 block:  13630998
Fetched v0 batches in range [13630000,13631000).
Fetch Config: Chain ID: 420.


span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [39]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

Merged v0 batches in range [13630000,13631000).


span batch tester `format`: Compare different tx encodings for span batch.


In [21]:
!$SPAN_BATCH_TESTER format --permutation 0123465 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[4866/4869] cnt: 3917, delta: 0, deltasum: 1700, originalCompressedSizeSum: 104922384
Reduction Percentage: 0.000000 %
[4867/4869] cnt: 3918, delta: 0, deltasum: 1700, originalCompressedSizeSum: 104924375
Reduction Percentage: 0.000000 %
[4868/4869] cnt: 3919, delta: 1, deltasum: 1701, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.034530 %
[4869/4869] cnt: 3920, delta: 1, deltasum: 1702, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.014959 %
Final Reduction Percentage: 0.001622 %


In [22]:
!$SPAN_BATCH_TESTER format --permutation 0134625 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[4866/4869] cnt: 3764, delta: 0, deltasum: -324, originalCompressedSizeSum: 104922384
Reduction Percentage: 0.000000 %
[4867/4869] cnt: 3765, delta: 0, deltasum: -324, originalCompressedSizeSum: 104924375
Reduction Percentage: 0.000000 %
[4868/4869] cnt: 3766, delta: 0, deltasum: -324, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.000000 %
[4869/4869] cnt: 3767, delta: 0, deltasum: -324, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.000000 %
Final Reduction Percentage: -0.000309 %


In [23]:
!$SPAN_BATCH_TESTER format --permutation 3041265 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.050697 %
[4866/4869] cnt: 2498, delta: -5, deltasum: -3059, originalCompressedSizeSum: 104922384
Reduction Percentage: -0.201613 %
[4867/4869] cnt: 2498, delta: -3, deltasum: -3062, originalCompressedSizeSum: 104924375
Reduction Percentage: -0.150678 %
[4868/4869] cnt: 2499, delta: 0, deltasum: -3062, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.000000 %
[4869/4869] cnt: 2500, delta: 0, deltasum: -3062, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.000000 %
Final Reduction Percentage: -0.002918 %


In [24]:
!$SPAN_BATCH_TESTER format --permutation 0125634 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[4866/4869] cnt: 4399, delta: 0, deltasum: 5422, originalCompressedSizeSum: 104922384
Reduction Percentage: 0.000000 %
[4867/4869] cnt: 4400, delta: 0, deltasum: 5422, originalCompressedSizeSum: 104924375
Reduction Percentage: 0.000000 %
[4868/4869] cnt: 4401, delta: 0, deltasum: 5422, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.000000 %
[4869/4869] cnt: 4402, delta: 0, deltasum: 5422, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.000000 %
Final Reduction Percentage: 0.005167 %


In [25]:
!$SPAN_BATCH_TESTER format --permutation 0324165 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[4866/4869] cnt: 3813, delta: -2, deltasum: 1338, originalCompressedSizeSum: 104922384
Reduction Percentage: -0.080645 %
[4867/4869] cnt: 3814, delta: 0, deltasum: 1338, originalCompressedSizeSum: 104924375
Reduction Percentage: 0.000000 %
[4868/4869] cnt: 3815, delta: 1, deltasum: 1339, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.034530 %
[4869/4869] cnt: 3816, delta: 2, deltasum: 1341, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.029918 %
Final Reduction Percentage: 0.001278 %


In [26]:
!$SPAN_BATCH_TESTER format --permutation 3041256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.050697 %
[4866/4869] cnt: 2572, delta: -5, deltasum: -5089, originalCompressedSizeSum: 104922384
Reduction Percentage: -0.201613 %
[4867/4869] cnt: 2572, delta: -3, deltasum: -5092, originalCompressedSizeSum: 104924375
Reduction Percentage: -0.150678 %
[4868/4869] cnt: 2572, delta: -1, deltasum: -5093, originalCompressedSizeSum: 104927271
Reduction Percentage: -0.034530 %
[4869/4869] cnt: 2573, delta: 0, deltasum: -5093, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.000000 %
Final Reduction Percentage: -0.004854 %


In [27]:
!$SPAN_BATCH_TESTER format --permutation 3401256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.050697 %
[4866/4869] cnt: 2620, delta: -1, deltasum: -4965, originalCompressedSizeSum: 104922384
Reduction Percentage: -0.040323 %
[4867/4869] cnt: 2620, delta: -3, deltasum: -4968, originalCompressedSizeSum: 104924375
Reduction Percentage: -0.150678 %
[4868/4869] cnt: 2620, delta: -1, deltasum: -4969, originalCompressedSizeSum: 104927271
Reduction Percentage: -0.034530 %
[4869/4869] cnt: 2621, delta: 0, deltasum: -4969, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.000000 %
Final Reduction Percentage: -0.004735 %


In [28]:
!$SPAN_BATCH_TESTER format --permutation 6051234 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.025349 %
[4866/4869] cnt: 3158, delta: -1, deltasum: 12045, originalCompressedSizeSum: 104922384
Reduction Percentage: -0.040323 %
[4867/4869] cnt: 3158, delta: -3, deltasum: 12042, originalCompressedSizeSum: 104924375
Reduction Percentage: -0.150678 %
[4868/4869] cnt: 3158, delta: -1, deltasum: 12041, originalCompressedSizeSum: 104927271
Reduction Percentage: -0.034530 %
[4869/4869] cnt: 3158, delta: -3, deltasum: 12038, originalCompressedSizeSum: 104933956
Reduction Percentage: -0.044877 %
Final Reduction Percentage: 0.011472 %


In [29]:
!$SPAN_BATCH_TESTER format --permutation 0132456 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[4866/4869] cnt: 4635, delta: -2, deltasum: -639, originalCompressedSizeSum: 104922384
Reduction Percentage: -0.080645 %
[4867/4869] cnt: 4636, delta: 0, deltasum: -639, originalCompressedSizeSum: 104924375
Reduction Percentage: 0.000000 %
[4868/4869] cnt: 4637, delta: 0, deltasum: -639, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.000000 %
[4869/4869] cnt: 4638, delta: 0, deltasum: -639, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.000000 %
Final Reduction Percentage: -0.000609 %


In [30]:
!$SPAN_BATCH_TESTER format --permutation 0352614 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.025349 %
[4866/4869] cnt: 3639, delta: 0, deltasum: -4662, originalCompressedSizeSum: 104922384
Reduction Percentage: 0.000000 %
[4867/4869] cnt: 3640, delta: 0, deltasum: -4662, originalCompressedSizeSum: 104924375
Reduction Percentage: 0.000000 %
[4868/4869] cnt: 3641, delta: 0, deltasum: -4662, originalCompressedSizeSum: 104927271
Reduction Percentage: 0.000000 %
[4869/4869] cnt: 3642, delta: 1, deltasum: -4661, originalCompressedSizeSum: 104933956
Reduction Percentage: 0.014959 %
Final Reduction Percentage: -0.004442 %
